# Chapter 11: Interfaces: From Protocols to ABCs

**Note:** This is a quite wordy chapter. The first half will be summarised very briefly.

The subject of this chapter is interfaces, from the dynamic protocols that characterise duck typing to abstract base classes (ABCs) that make interfaces explicit and verify implementations for conformance.

It is useful to reiterate the idea of *duck typing* when exploring interfaces:

> Duck typing ignores an objects actual type, focusing instead on ensuring that the object implements the method names, signatures, and semantics required for its intended use.

## Interfaces and Protocols in Python Culture

Every class has an interface: the set public attributes (methods or data attributes) implemented or inherited by the class.

An interface can be viewed as a set of methods to fulfill a role is what is known as a *protocol*. Protocols are independent of inheritance. A class may implement several protocols, enabling its instances to fulfill several roles.

Because protocols are informal, the cannot be enforced like formal interfaces can.

By definition, private and protected attributes are not part of an interface, even if "protected" is merely a naming convention.

On the other hand, public data attributes can be part of the interface of an object because a data attribute can always be turned into a properting implementing getter/setter logic.

A useful complementary definition of an interface is: the subset of an object's public methods that enable it to play a specific role in the system.

This is what is implied when the Python documentation mentions a "file-like object" or "an iterable".

## Python Digs Sequences

The Python data model is designed to cooperate with essential protocols as much as possible.

One of the most fundamental Python protocols is the sequence protocol. The UML diagram below shows how the formal `Sequence` interface is implemented as an ABC:

![](../figs/uml-mutablesequence.png)

However, given the importance of the sequence protocol, in the abscence of `__iter__` and `__contains__` methods, Python can still make iteration and the `in` operator work by invoking `__getitem__`.


## Monkey-Patching to Implement a Protocol at Runtime

The `FrenchDeck` class from example 11-4 cannot be shuffled because it does not support item assignment, because it implements the *immutable* sequence protocol.

Mutable sequences must also provide a `__setitem__` method.

Because Python is dynamic, we can fix this at runtime. The following shows an example of *monkey-patching*, which means changing a class or module at runtime.

In [1]:
from examples.deck import FrenchDeck
from random import shuffle

deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [2]:
# Example 11-6
def setitem(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = setitem
shuffle(deck)
deck[:5]

[Card(rank='8', suit='clubs'),
 Card(rank='4', suit='hearts'),
 Card(rank='7', suit='hearts'),
 Card(rank='6', suit='diamonds'),
 Card(rank='Q', suit='spades')]

The above example highlights that protocols are dynamic: `random.shuffle` doesn't care about what type of argument it gets, in only needs the object to implement part of the mutable sequence protocol.

## Alex Martellis Waterfowl

This section presents a guest essay by Alex Martelli, explaining why he believes the addition of ABCs is a good thing in Python. See **pp. 326-329**. The essay is a bit lofty, but I think the main takeaways are:

* The use of `isinstance` and `issubclass` is more acceptable when testing against ABCs. Otherwise they are a [code smell](https://en.wikipedia.org/wiki/Code_smell). StackOverflow [explains](https://stackoverflow.com/questions/13636149/is-using-python-isinstance-ever-right) [why](https://stackoverflow.com/questions/13636149/is-using-python-isinstance-ever-right).

* Duck typing is usually simpler and more flexible than type checks. In practice this means wrapping code in `try`/`except` blocks (`try` something assuming some behaviour, catch any errors with `except`).

* Use restraint when creating ABCs. They are meant to encapsulate very general concepts and abstractions introduced by a framework. Readers usually just need to use existing ABCs correctly, to get 99.9% of the benefits without serious risk of misdesign.

* Martelli coins the word *goose typing*:

> What goose typing means is: `isinstance(obj, cls)` is now just fine... as long as `cls` is an Abstract Base Class.

## Subclassing an ABC

The following example leverages an existing ABC, `collections.MutableSequence`, to create a new version of `FrenchDeck`.

In [19]:
# Example 11-8. frenchdeck2.py: FrenchDeck2, a subclass of collections.MutableSequence

import collections

Card = collections.namedtuple("Card", ["rank", "suit"])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(11)] + list("JQKA")
    suits = "spades diamonds clubs hearts".split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in suits for rank in ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    # Implemented to allow shuffling
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    # Enforced by collections.MutableSequence
    def __delitem__(self, position):
        del self._cards[position]
        
    # Enforced by collections.MutableSequence
    def insert(self, position, value):
        self._cards.insert(position, value)
        

Python does not check for the implementation of the abstract methods at import time (when module is loaded and compiled), only at runtime.

However, when we try to instantiate an instance of `FrenchDeck2`, **we will get a `TypeError` if we have failed to implement any of the abstract methods** (`__delitem__` and `insert`).

From `Sequence` (see figure in **Python Digs Sequences** section), `FrenchDeck2` inherits the following ready-to-use concrete methods (also called "mixin methods"):

* `__contains__`

* `__iter__`

* `__reversed__`

* `index`

* `count`

From `MutableSequence` it gets:

* `append`

* `reverse`

* `extend`

* `pop`

* `remove`

* `__iadd__`

The concrete methods in each `collections.abc` ABC are implemented in terms of the public interface of the class.

They work without any knowledge of the internal structure of the instances.

**Note:** Methods inherited from ABCs can be overwritten with more efficient implentations (such as writing a more efficient `__contains__` if your concrete sequence keeps its items sorted, using `bisect`).

## ABCs in the Standard Library

Most ABCs are defined in `collections.abc`, but some are also in the `numbers` and `io` packages.

Note that there are two `abc` modules in the standard library, the other is is where `abc.ABC` is defined. Every ABC depends on it, but we don't need to import it except to create our own ABC.

The following figure is a summary UML class diagram (without attribute names) of all 16 [ABCs defined in](https://docs.python.org/3/library/collections.abc.html) `collections.abc` as of Python 3.4:

![](../figs/uml-abc.png)

#### `Iterable`, `Container`, and `Sized`

Every collection should either inherit from the ABCs or at least implement compatible protocols.

#### `Sequence`, `Mapping`, and `Set`

These are the main immutable collection types, and each has a mutable subclass

#### `MappingView`

The objects returned from mapping methods `.items()`, `keys()` and `.values()` inherit from `ItemsView`, `KeysView`, and `ValuesView` respectively. The first two also inherit the interface of `Set`.

#### `Callable` and `Hashable`

Their main use is to support the `isinstance` built-in as a safe way of determining whether an object is callable or hashable.

#### `Iterator`

Discussed further in [Chapter 14](). Not that `Iterator` subclasses `Iterable`.



## The Numbers Tower of ABCs

The `numbers` package defines the linear hierarchy of of ABCs, where `Numbers` is the topmost superclass, `Complex` is it's immediate subclass, and so on, down to `Integral`:

* `Number`

* `Complex`

* `Real`

* `Rational`

* `Integral`

## Defining and Using an ABC

In the following example we will create an ABC. 

**Problem statement:** Assume we are creating an ad management framework called `ADAM`. One of its requirements is to support user-provided nonrepeating random-picking classes. To make it clear to `ADAM` users what is expected of a "nonrepeating random-picking" component, we'll define an ABC.

We will use real-world metaphors (bingo cages and lottery blowers) to name our ABCs, to get across the point that we are picking items from a finite set, without repeating, until the set is exhausted. The ABC will be named `Tombola`, after the Italian name of bingo and the tumbling container it uses to mix number.

![](../figs/tombola.png)

In [ ]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable"""
        
    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.

        This method should raise 'LookupError' when the instance is empty.
        """
        
    def loaded(self):
        """Return 'True' if there's at least 1 item, 'False' otherwise."""
        
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

#### Main takeaways regarding about the above implementation:

* An abstract method is marked with the `@abstractmethod` decorator, and often its body is empty except for a docstring.

* An abstract method can have an implementation, but subclasses will be forced to override it. However, they will be able to invoke the abstract method using `super()`, adding functionality instead of implementing it from scratch.

* It's OK to implement concrete methods in ABCs, as long as they only depend on the other methods in the interface.

#### Other things to note:

* The `LookupError` may be raised, but there is no way to declare this in Python, except in the documenation.

* `LookupError` was chosen because of it's place in the Python hierarchy of exceptions in relation to `IndexError` and `KeyError`, which are the most likely to be raised by `Tombola`. See the [documentation](https://docs.python.org/3/library/exceptions.html#exception-hierarchy) for the complete tree.

In the following example we create a `Fake` class to see how ABCs enforce interfaces.

In [7]:
# Example 11-11. A fake Tombola doesn't go undetected.

from examples.tombola import Tombola

class Fake(Tombola):
    def pick(self):
        return 13

print(repr(Fake), type(Fake))
f = Fake()

<class '__main__.Fake'> <class 'abc.ABCMeta'>


TypeError: Can't instantiate abstract class Fake with abstract methods load

### ABC Syntax Details

**Note:** There are some changes to more recent versions of Python. Check the [documentation](https://docs.python.org/3/library/abc.html#abc.ABC).

The best way to declare an ABC is to subclass `abc.ABC` or any other ABC.

In [9]:
import abc
help(abc.ABC)

Help on class ABC in module abc:

class ABC(builtins.object)
 |  Helper class that provides a standard way to create an ABC using
 |  inheritance.
 |  
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()



It is possible to stack decorators on top of `@abstractmethod`. 

The order of stacked function decorators usually matters. When `abstractmethod()` is applied in combination with other method descriptors, it **should be applied as the innermost decorator**.

### Subclassing the Tombola ABC

We will develop two concrete subclasses that satisfy the `Tombola` ABC interface (see the figure in **Defining and Using an ABC**).

`BingoCage`, which implements a variation of example 5-8 using a better randomiser. It overrides `inspect` and adds `__call__`.

In [13]:
# Example 11-12. BingoCage is a concrete subclass of Tombola.

import random
from examples.tombola import Tombola


class BingoCage(Tombola):
    def __init__(self, items):
        self._randomiser = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomiser.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError("pick from empty BingoCage")

    def __call__(self):
        self.pick()
        

The methods inherited from `Tombola` are not as fast as they could be for `BingoCage`. The point is to illustrate that we can lazily inherit suboptimal concrete methods from an ABC.

The following example shows a different but equally valid implementation of the `Tombola` interface.

In [15]:
# Example 11-13. LotteryBlower is a concrete subclass that overrides the inspect 
# and loaded methods from Tombola.

import random
from examples.tombola import Tombola

class LotterBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError("pick from empty BingoCage")
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))
    

#### Things to not about the above implementation

* `random.randrange(...)` raises a `ValueError` if the range is empty, which we catch and throw a `LookupError` instead to be compatible with `Tombola`.

* We override `loaded` to avoid calling `inspect`.

* In `__init__`, `self._balls` is not just a reference to `iterable`, but stores it as a `list`. This makes the class more flexible, as it can now accept any iterable as an argument.

* `self._balls = list(iterable)` also grants access to the `pop` method, and it stores the argument as a copy. This is good practice if the input is a `list`, since we are removing items and the user may not anticipate that the provided `list` of items may change.

## A Virtual Subclass of Tombola

An essential characteristic of goose typing is the ability to register a class as a *virtual subclass* of an ABC, even if it does not inherit from it.

This is done by calling the `register` method on the ABC.

**The registered class then becomes a virtual subclass of the ABC, and will be recognised as such by functions like `issubclass` and `isinstance`, but will not inherit any methods or attributes from the ABC**.

See this [StackOverflow thread](https://stackoverflow.com/questions/51666120/whats-the-usage-of-a-virtual-subclass) for intuition on why virtual subclasses are useful.

In [16]:
# Example 11-14. class TomboList is a virtual subclass of Tombola

from random import randrange
from examples.tombola import Tombola

@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self: 
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError("pop from empty TomboList")
    
    # TomboList.load is the same as list.extend
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    

#### Things to not about the above implementation

* Because of the registration, `issubclass` and `isinstance` act as if `TomboList`is a subclass of `Tombola`.

* Inheritance is guided by a special class attribute names `__mro__` (**m**ethod **r**esolution **o**rder), which lists the class and its superflasses in the order Python uses to search for methods. `Tombola` is not in this list.

In [17]:
TomboList.__mro__

(__main__.TomboList, list, object)

There are details on **pp. 347-350** on Ramalho's use of `__subclasses__()` and `_abc_registry` to write tests for the above implementations.

## Usage of `register` in Practice

The above example uses the decorator syntax to register the virtual subclass.

In practice it is more widely deployed as a function to register classes defined elsewhere. See for example the [source code](https://github.com/python/cpython/blob/3.8/Lib/_collections_abc.py) for `collections.abc`

**Example of virtual subclasses of `Sequence`.** The built-in types `tuple`, `str`, `range` and `memoryview` are registered as virutal subclasses like this:

```python
Sequence.register(tuple)
Sequence.register(str)
Sequence.register(range)
Sequence.register(memoryview)
```

## Geese Can Behave as Ducks

Some ABCs, such as `abc.Sized`, implement a special class method named `__subclasshook__`, which adds some duck typing DNA to the goose typing proposition.

In the case of `abc.Sized`, this means that any class that defines the `__len__` special method will be considered a virtual subclass of `abc.Sized` by `issubclass` and `isinstance`.

In other words, `__subclasshook__` allows classes to be recognised as virtual subclasses even without explicit registration.

**Note:** It is usually not a good idea to implement your own `__subclasshook__`. See **pp. 351-352** for more details.